In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "shafieiali666"
os.environ['KAGGLE_KEY'] = "2a72371e7748b1c2a39075b67b207030"

In [ ]:
!kaggle datasets download -d sajjadayobi360/flickrfa

100% 3.50G/3.50G [03:08<00:00, 12.4MB/s]
100% 3.50G/3.50G [03:08<00:00, 20.0MB/s]


In [ ]:
! unzip flickrfa.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/4946968361.jpg   
  inflating: images/4946977529.jpg   
  inflating: images/4946986337.jpg   
  inflating: images/4947072825.jpg   
  inflating: images/4947153201.jpg   
  inflating: images/4947321869.jpg   
  inflating: images/4947861901.jpg   
  inflating: images/494792770.jpg    
  inflating: images/4948224114.jpg   
  inflating: images/494823012.jpg    
  inflating: images/4948237706.jpg   
  inflating: images/49482602.jpg     
  inflating: images/4948403203.jpg   
  inflating: images/4948424467.jpg   
  inflating: images/4948440838.jpg   
  inflating: images/4948441067.jpg   
  inflating: images/4948635454.jpg   
  inflating: images/4948865477.jpg   
  inflating: images/4948875390.jpg   
  inflating: images/4948910881.jpg   
  inflating: images/4948943251.jpg   
  inflating: images/4948995543.jpg   
  inflating: images/494907021.jpg    
  inflating: images/494921598.jpg    
  inflating: images/4949430092.jpg   

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git 
! pip install transformers 
! pip install arabic-reshaper
! pip install python-bidi 

In [ ]:
from __future__ import unicode_literals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import time
import copy
import PIL
import torch
import os
import clip
import requests
from PIL import Image
import torch.nn as nn
import multiprocessing
from tqdm import tqdm
from io import BytesIO
import torchvision.transforms as transforms
from torch.cuda.amp import autocast
from transformers import CLIPModel, CLIPConfig, CLIPVisionModel, CLIPFeatureExtractor
from transformers import AutoModel, AutoTokenizer, AutoModel, TFAutoModel, AutoConfig
from transformers import BertModel
from transformers import TrainingArguments, Trainer, RobertaModel
from transformers import default_data_collator
from IPython.display import clear_output


In [ ]:
df = pd.read_csv('dataset.csv')
df = df.dropna()
df = df.reset_index(drop=True)
# df=df[:100]
print(df.shape)
df.head()

(32382, 2)


,caption,image
0,دو مرد با پیراهن سبز در حیاط ایستاده اند.,1000092795.jpg
1,سه مرد روی یک دکل بزرگ.,10002456.jpg
2,دختر بچه ای در حال بالا رفتن از یک خانه بازی چوبی,1000268201.jpg
3,مردی روی نردبان پنجره ای را تمیز می کند,1000344755.jpg
4,دو مرد در حال پختن غذا هستند.,1000366164.jpg


In [ ]:
images_list_final = np.array([f"images/{i}" for i in df['image']])
text_list_final = np.array([i for i in df['caption']])

In [ ]:
from bidi.algorithm import get_display
from arabic_reshaper import reshape
fig,axs=plt.subplots(3,1,figsize=(15,12))
counter=0
while counter<3:
  index=np.random.randint(len(images_list_final))
  axs[counter%3].imshow(plt.imread(images_list_final[index]))
  axs[counter%3].title.set_text(get_display(reshape(text_list_final[index])))
  counter+=1


## Model

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
model, preprocess = clip.load("ViT-L/14", device=device, jit=False)
model = model.float()
parsbert_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(parsbert_path)
tokenizer = AutoTokenizer.from_pretrained(parsbert_path)
parsbert = BertModel.from_pretrained(parsbert_path).float().to(device)

100%|███████████████████████████████████████| 890M/890M [00:13<00:00, 68.0MiB/s]


Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

In [ ]:
def my_encode_text(text):
    return parsbert(text).pooler_output
model.encode_text = my_encode_text
model.transformer = parsbert

In [ ]:
parsbert.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [ ]:
for i in range(df.shape[0]):
  df.iloc[i]["image"]="images/"+df.iloc[i]["image"]
df.head()

,caption,image
0,دو مرد با پیراهن سبز در حیاط ایستاده اند.,images/1000092795.jpg
1,سه مرد روی یک دکل بزرگ.,images/10002456.jpg
2,دختر بچه ای در حال بالا رفتن از یک خانه بازی چوبی,images/1000268201.jpg
3,مردی روی نردبان پنجره ای را تمیز می کند,images/1000344755.jpg
4,دو مرد در حال پختن غذا هستند.,images/1000366164.jpg


In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
BATCH_SIZE = 64

class ImageCaptionDataset(Dataset):
    def __init__(self, all_images, all_captions):
        self.all_images = all_images
        self.all_captions=[tokenizer.encode_plus(text,max_length=77,truncation=True,add_special_tokens=True,
                                         return_token_type_ids=True,return_attention_mask=True,
                                         padding='max_length',return_tensors='pt',)['input_ids'].numpy().ravel()
                                          for text in all_captions]
        self.all_captions=torch.from_numpy(np.array(self.all_captions)).to(device)

    def __len__(self):
        return len(self.all_captions)

    def __getitem__(self, index):
        image = preprocess(Image.open(self.all_images[index]))
        caption = self.all_captions[index]
        return image, caption

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.15, shuffle=True, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.15, shuffle=True, random_state=42)

train_dataset = ImageCaptionDataset(train_df['image'].tolist(), train_df['caption'].tolist())
test_dataset = ImageCaptionDataset(test_df['image'].tolist(), test_df['caption'].tolist())
val_dataset = ImageCaptionDataset(val_df['image'].tolist(), val_df['caption'].tolist())
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE)
test_dataloader= DataLoader(test_dataset, batch_size = BATCH_SIZE)
val_dataloader= DataLoader(val_dataset, batch_size = BATCH_SIZE)

In [ ]:
EPOCH = 10
LR = 1e-7
EPS = 1e-9
WEIGHT_DECAY = 0.1
MAX_LR = 1e-2
BASE_MODEL_PATH = 'drive/MyDrive/Nlp_HW3/models/'

In [ ]:
loss_img = nn.CrossEntropyLoss()
loss_captions = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=LR,
                             betas=(0.9,0.98),
                             eps=EPS,
                             weight_decay=WEIGHT_DECAY) 

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                                max_lr=MAX_LR, 
                                                steps_per_epoch=len(train_dataloader), 
                                                epochs=EPOCH)

In [ ]:
layer_num = [0, 0]
freeze_nlp_layer_thr = 20
freeze_vision_layer_thr=20
for param in model.transformer.parameters():
    layer_num[0]+=1
for param in model.visual.parameters():
    layer_num[1]+=1

for i, param in enumerate(model.transformer.parameters()):
    if freeze_nlp_layer_thr >= layer_num[0] - i:
        param.requires_grad = True
    else:
        param.requires_grad = False
for i, param in enumerate(model.visual.parameters()):
    if freeze_vision_layer_thr >= layer_num[1] - i:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [ ]:
def train_model(model,optimizer,scheduler, current_epoch,num_epochs=10):
  for epoch in  range(current_epoch,num_epochs):
    print(f"Epoch {epoch}/{num_epochs}")
    print()

    print("training ....")
    train_loss=0
    iter_cnt=0
    for batch in tqdm(train_dataloader):
      images,captions=batch
      images=images.to(device)
      captions=captions.to(device)
      optimizer.zero_grad()
      image_output, caption_output = model(images, captions)
      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
      batch_loss = (loss_img(image_output,ground_truth) + loss_captions(caption_output,ground_truth))/2
      batch_loss.backward()
      optimizer.step()
      train_loss+=batch_loss
      iter_cnt+=1
    epoch_loss=train_loss/iter_cnt
    scheduler.step()
    print(f'Train Loss: {epoch_loss:.4f}')
    torch.save({'epoch': EPOCH,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'scheduler_state_dict':scheduler.state_dict(),'loss': epoch_loss,}, BASE_MODEL_PATH+f'checkpoint_ep{epoch}.pt')
# -------------------------------------------------------------------------------------------------------------------------
    print("validation ....")
    val_loss=0
    iter_cnt=0
    for batch in tqdm(val_dataloader):
      images,captions=batch
      images=images.to(device)
      captions=captions.to(device)  
      optimizer.zero_grad()
      with torch.no_grad():
        image_output, caption_output = model(images, captions)
        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        batch_loss = (loss_img(image_output,ground_truth) + loss_captions(caption_output,ground_truth))/2
      val_loss+=batch_loss
      iter_cnt+=1
    epoch_loss=val_loss/iter_cnt
    print(f'Validation Loss: {epoch_loss:.4f}')
    
                    


In [ ]:
train_model(model=model, optimizer=optimizer, scheduler=scheduler, current_epoch=0,num_epochs=EPOCH)